# RECALL @ K
Мы разрабатываем платформу Greenterest для хранения картинок, сгенерированных нейросетями.

Мы строим поисковой сервис, который по запросу пользователя выдаёт 20+ релевантных изображений.

Модель выдаёт меру релевантности картинок запросу.

Чтобы оценить результаты модели, мы должны посмотреть на историю: какие были запросы в прошлом, какой SERP (Search Engine Result Page) был у старого алгоритма, на какие из картинок пользователь кликнул и т.д. В качестве таргета мы можем взять исторические реакции пользователей, а также проверить, как переранжировал бы картинки новый алгоритм. Будут ли в топ-20 более релевантные картинки, чем у старой системы, или нет?

Мы реализуем 4 метрики, которые помогут нам оценить качество нашей модели во время оффлайн тестирования (т.е. до того, как раскатывать поиск на реальных пользователей):

Recall @ K
Precision @ K
F1-Score @ K
Specificity @ K

Для примера, стандартный Recall в классификации сообщает, какая доля объектов положительного класса (в нашем случае, кликнутых картинок) включена моделью как положительный класс. Очевидно, для максимального Recall мы могли бы включить в положительный класс все картинки, но особого смысла от такой модели не будет. С другой стороны, тогда просядет Precision (сколько из рекомендованных картинок действительно были кликнуты). 

Чтобы избавиться от этой неопределённости, в ранжировании, поиске и рекомендациях используют метрики по типу Recall @ K, которые оперируют с предсказанным скором релевантности и топ-K от него сравнивают с положительным классом. Это довольно естественная метрика, потому что редко кто листает дальше первых 10-20 картинок. Соответственно, в топ-5 или топ-10 попадёт то, что пользователь точно увидит.

Метрика Recall @ K (полнота при K) измеряет, сколько релевантных элементов из всех возможных попали в топ-K предложений (т.е. в первые K элементов). Она отражает, насколько хорошо ваша модель захватывает релевантные объекты среди K лучших результатов.

Precision @ K (точность при K) измеряет долю правильных (релевантных) предложений среди первых K элементов, предложенных моделью. Эта метрика показывает, насколько точны топ-K предсказания: среди предложенных результатов, сколько из них действительно релевантны.


 F1-Score @ K:
F1-Score — это гармоническое среднее между Precision и Recall, которое дает сбалансированную оценку точности и полноты. Используется, когда важно учитывать как пропущенные релевантные элементы (Recall), так и ложные срабатывания (Precision).

 Specificity @ K:
Specificity (специфичность) — это метрика, которая показывает долю правильно предсказанных нерелевантных элементов среди всех нерелевантных элементов. Она используется в задачах классификации, но в случае рекомендаций и ранжирования ее можно адаптировать для анализа топ-K предсказаний.

Пояснение:
True Negatives (TN) — это нерелевантные элементы, которые не были предложены моделью (модель правильно их "отвергла").
False Positives (FP) — это нерелевантные элементы, которые были предложены моделью (модель ошибочно их "одобрила").

In [ ]:
from typing import List


def recall_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Return the recall"""
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    scores_k = sorted_indices[:k]
    relevant_elements = [i for i, label in enumerate(labels) if label == 1]
    relevant_at_k = set(scores_k) & set(relevant_elements)
    return len(relevant_at_k) / len(relevant_elements) if len(relevant_elements) > 0 else 0.0


def precision_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Return the precision"""
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    scores_k = sorted_indices[:k]
    relevant_elements = [i for i, label in enumerate(labels) if label == 1]
    relevant_at_k = set(scores_k) & set(relevant_elements)
    return len(relevant_at_k) / k if k > 0 else 0.0


def specificity_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """ Specify at k"""
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    top_k_indices = sorted_indices[:k]

    # Определяем нерелевантные элементы (где label == 0)
    non_relevant_indices = [i for i, label in enumerate(labels) if label == 0]

    # Считаем True Negatives: нерелевантные элементы, которые не попали в топ-K
    true_negatives_at_k = len(set(non_relevant_indices) - set(top_k_indices))

    # Общее количество нерелевантных элементов
    total_non_relevant = len(non_relevant_indices)

    if total_non_relevant == 0:
        return 0.0  # Во избежание деления на ноль

    return true_negatives_at_k / total_non_relevant


def f1_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Returns the f1 at k"""
    p_k = precision_at_k(labels, scores, k)
    r_k = recall_at_k(labels, scores, k)
    return 2 * p_k * r_k / (p_k + r_k) if (p_k + r_k) != 0.0 else 0.0

In [1]:
from typing import List

import numpy as np


def recall_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Compute recall at k.

    Args:
        y_true: list of true labels
        y_pred: list of predicted labels
        k: number of top labels to consider

    Returns:
        Recall at k
    """

    # Compute all rates
    positive_class = np.argsort(scores)[::-1][:k]
    negative_class = np.argsort(scores)[::-1][k:]
    tp_rate = np.sum([labels[i] == 1 for i in positive_class])
    fn_rate = np.sum([labels[i] == 1 for i in negative_class])

    # Compute recall at k
    recall = tp_rate / (tp_rate + fn_rate)

    return recall


def precision_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Compute precision at k.

    Args:
        y_true: list of true labels
        y_pred: list of predicted labels
        k: number of top labels to consider

    Returns:
        Precision at k
    """

    # Compute all rates
    positive_class = np.argsort(scores)[::-1][:k]
    tp_rate = np.sum([labels[i] == 1 for i in positive_class])
    fp_rate = np.sum([labels[i] == 0 for i in positive_class])

    # Compute precision at k
    precision = tp_rate / (tp_rate + fp_rate)

    return precision


def specificity_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Compute specificity at k.

    Args:
        y_true: list of true labels
        y_pred: list of predicted labels
        k: number of top labels to consider

    Returns:
        Specificity at k
    """

    # Compute all rates
    positive_class = np.argsort(scores)[::-1][:k]
    negative_class = np.argsort(scores)[::-1][k:]

    tn_rate = np.sum([labels[i] == 0 for i in negative_class])
    fp_rate = np.sum([labels[i] == 0 for i in positive_class])

    # Compute specificity at k
    specificity = tn_rate / ((tn_rate + fp_rate) or 1e-16)

    return specificity


def f1_at_k(labels: List[int], scores: List[float], k=5) -> float:
    """Compute f1 score at k.

    Args:
        y_true: list of true labels
        y_pred: list of predicted labels
        k: number of top labels to consider

    Returns:
        F1 score at k
    """

    # Compute precision and recall at k
    precision = precision_at_k(labels, scores, k)
    recall = recall_at_k(labels, scores, k)

    # Compute f1 score at k
    f1_score = 2 * precision * recall / ((precision + recall) or 1e-16)

    return f1_score
